In [ ]:
model=SSCDNonLModel99999999_(class_count,n_bands,150)


In [23]:
class SSCDNonLModel(nn.Module):
    def __init__(self, num_classes, n_bands, chanel):
        super(SSCDNonLModel, self).__init__()
        #self.num_Node=num_Node
        self.bands=n_bands
        chanel=chanel
        kernel=5
        CCChannel=25
        # 第一层
        self.b1=nn.BatchNorm2d(self.bands)
        self.con1=nn.Conv2d(self.bands, chanel, 1, padding=0,bias=True)
        self.s1=nn.Sigmoid()
        self.cond1=nn.Conv2d(chanel, chanel, kernel, padding=2, groups=chanel, bias=True)
        self.sd1=nn.Sigmoid()

        # 第二层
        self.b2=nn.BatchNorm2d(self.bands+chanel)
        #self.nlcon1=NonLocalBlock(300, 300, True)
        #self.gcn1=GCNtrans(300,1000)
        #self.bcat=nn.BatchNorm2d(300+300)
        self.con2=nn.Conv2d(self.bands+chanel, chanel, 1, padding=0,bias=True)
        self.s2=nn.Sigmoid()
        self.cond2=nn.Conv2d(chanel, CCChannel, kernel, padding=2, groups=25, bias=True)
        self.sd2=nn.Sigmoid()
        
        #对齐维度
        self.b4=nn.BatchNorm2d(CCChannel)
        # 条状池化
        self.nlcon2=CrissCrossAttention(CCChannel)
        self.nlcon3=CrissCrossAttention(CCChannel)
        #连接特征
        self.bcat=nn.BatchNorm2d(CCChannel+CCChannel)
        # 第三层
        # 拼接第二层输出和cc输出
        self.con4=nn.Conv2d(CCChannel+CCChannel, chanel, 1, padding=0, bias=True)
        self.s4=nn.Sigmoid()
        self.cond4=nn.Conv2d(chanel, chanel, kernel, padding=2, groups=chanel, bias=True)
        self.sd4=nn.Sigmoid()
        
        第四层
        self.b5=nn.BatchNorm2d(CCChannel+chanel)
        self.con5=nn.Conv2d(CCChannel+chanel, chanel, 1, padding=0, bias=True)
        self.s5=nn.Sigmoid()
        self.cond5=nn.Conv2d(chanel, chanel, kernel, padding=2, groups=chanel, bias=True)
        self.sd5=nn.Sigmoid()

        #self.b6=nn.BatchNorm2d(300+300)
        self.con6=nn.Conv2d(chanel+CCChannel, num_classes+1, 1, padding=0, bias=True)



    def forward(self, x):
        n = x.size(0)
        H=x.size(2)
        W=x.size(3)

        out1=self.b1(x)
        out1=self.con1(out1)
        out1=self.s1(out1)
        out1=self.cond1(out1)
        out1=self.sd1(out1)

        # 拼接特征，对齐维度
        out2=torch.cat((out1,x),1)
        out2=self.b2(out2)
        out2=self.con2(out2)
        out2=self.s2(out2)
        out2=self.cond2(out2)
        out2=self.sd2(out2)

        
        xx=self.b4(out2)
        #循环cc
        nl2=self.nlcon2(xx)
        nl2=self.nlcon2(nl2)
        nl3=self.nlcon3(xx)
        nl3=self.nlcon3(nl3)
        #得到输出
        nl2=(nl2+nl3)*0.7+xx
        out4=torch.cat((xx, nl2),1)
        #拼接
        out4=self.bcat(out4)
        out4=self.con4(out4)
        out4=self.s4(out4)
        out4=self.cond4(out4)
        out4=self.sd4(out4)



        out5=torch.cat((out4,out2),1)
        out5=self.b5(out5)

        out5=self.con5(out5)
        out5=self.s5(out5)
        out5=self.cond5(out5)
        out5=self.sd5(out5)


        out6=torch.cat((out5,out2),1)
        out6=self.con6(out6)

        return out6

In [22]:
import tensorwatch as tw
import torchvision.models

tw.model_stats(model,[1, 200,145, 145])

RuntimeError: running_mean should contain 650 elements not 50

In [10]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch.utils.data as pydata
from torchsummary import summary
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import random
from matplotlib import cm
import spectral as spy
from sklearn import metrics
from sklearn import preprocessing
import time
from torch.autograd import Variable
from H_datapy import *

#import CCnet2BModel1 as CC2B
#import CCnetConcat2B1 as CCCon2B
#from CCnetConcat2B1 import SSCDNonLModel_gcn
import torch.nn.functional as F
from autis import *

In [11]:

from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv

In [12]:
import torch.nn as nn
from torch.nn import Softmax
class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        #self.activation = activation
 
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        #self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))
 
        self.softmax  = nn.Softmax(dim=-1)
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N)
        #proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N
 
        #out = torch.bmm(proj_value,attention.permute(0,2,1) )
        #out = out.view(m_batchsize,C,width,height)
 
        #out = self.gamma*out + x
        #return out,attention
        return attention


In [13]:
class SSCDNonLModel_gcn(nn.Module):
    def __init__(self, num_classes, n_bands, chanel):
        super(SSCDNonLModel_gcn, self).__init__()
        #self.num_Node=num_Node
        self.bands=n_bands
        chanel=chanel
        kernel=5
        CCChannel=25

        self.b1=nn.BatchNorm2d(self.bands)
        self.con1=nn.Conv2d(self.bands, chanel, 1, padding=0,bias=True)
        self.s1=nn.Sigmoid()
        self.cond1=nn.Conv2d(chanel, chanel, kernel, padding=2, groups=chanel, bias=True)
        self.sd1=nn.Sigmoid()


        self.b2=nn.BatchNorm2d(self.bands+chanel)
        #self.nlcon1=NonLocalBlock(300, 300, True)
        #self.gcn1=GCNtrans(300,1000)
        #self.bcat=nn.BatchNorm2d(300+300)
        self.con2=nn.Conv2d(self.bands+chanel, chanel, 1, padding=0,bias=True)
        self.s2=nn.Sigmoid()
        self.cond2=nn.Conv2d(chanel, CCChannel, kernel, padding=2, groups=25, bias=True)
        self.sd2=nn.Sigmoid()

        self.b4=nn.BatchNorm2d(CCChannel)

        # GCN
        #self.conv1 = GCNConv(dataset.num_features, 16, cached=True,normalize=not args.use_gdc)
        self.att_map=Self_Attn(CCChannel)
        self.conv1=GCNConv(1,int(CCChannel/2),cached=True)
        self.conv2=GCNConv(int(CCChannel/2),CCChannel,cached=True)

        # end
        #self.b4=nn.BatchNorm2d(CCChannel)
        #self.nlcon2=CrissCrossAttention(CCChannel)
        #self.nlcon3=CrissCrossAttention(CCChannel)
        self.bcat=nn.BatchNorm2d(CCChannel+CCChannel)
        self.con4=nn.Conv2d(CCChannel+CCChannel, chanel, 1, padding=0, bias=True)
        self.s4=nn.Sigmoid()
        self.cond4=nn.Conv2d(chanel, chanel, kernel, padding=2, groups=chanel, bias=True)
        self.sd4=nn.Sigmoid()

        self.b5=nn.BatchNorm2d(CCChannel+chanel)
        self.con5=nn.Conv2d(CCChannel+chanel, chanel, 1, padding=0, bias=True)
        self.s5=nn.Sigmoid()
        self.cond5=nn.Conv2d(chanel, chanel, kernel, padding=2, groups=chanel, bias=True)
        self.sd5=nn.Sigmoid()

        #self.b6=nn.BatchNorm2d(300+300)
        self.con6=nn.Conv2d(chanel+CCChannel, num_classes+1, 1, padding=0, bias=True)



    def forward(self, x):
        n = x.size(0)
        H=x.size(2)
        W=x.size(3)

        out1=self.b1(x)
        out1=self.con1(out1)
        out1=self.s1(out1)
        out1=self.cond1(out1)
        out1=self.sd1(out1)

        out2=torch.cat((out1,x),1)
        out2=self.b2(out2)
        out2=self.con2(out2)
        out2=self.s2(out2)
        out2=self.cond2(out2)
        out2=self.sd2(out2)


        xx=self.b4(out2)
        att_map=self.att_map(xx)
        adj=att_map
        adj=adj.view(21025,21025)
        adj[adj<0.5]=0
        adj[adj>=0.5]=1
        edge_index=adj.nonzero().t().contiguous()
        
        out_gcn1=self.conv1(xx.view(-1,1),edge_index)
        #out_gcn1=self.conv1(out_finalconv.view(-1, 1),edge_index)
        out_gcn1 = F.relu(out_gcn1)
        out_gcn1 = F.dropout(out_gcn1, training=self.training)
        out_gcn2=self.conv2(out_gcn1,edge_index)
        #x=F.log_softmax(out_gcn2, dim=1)
        #print(out_gcn2.shape)

        out_gcn2=out_gcn2.view(1,-1,H,W)  


        #nl2=(nl2+nl3)*0.7+xx
        out4=torch.cat((xx, out_gcn2),1)
        out4=self.bcat(out4)
        out4=self.con4(out4)
        out4=self.s4(out4)
        out4=self.cond4(out4)
        out4=self.sd4(out4)



        out5=torch.cat((out4,out2),1)
        out5=self.b5(out5)

        out5=self.con5(out5)
        out5=self.s5(out5)
        out5=self.cond5(out5)
        out5=self.sd5(out5)


        out6=torch.cat((out5,out2),1)
        out6=self.con6(out6)

        return out6

In [14]:
#from functions import CrissCrossAttention

samples_type=['ratio','same_num'][0]

In [15]:
FLAG =1
curr_train_ratio=0.1
OA_ALL = []
AA_ALL = []
KPP_ALL = []
AVG_ALL = []
Seed_List=[]

In [16]:
    if FLAG == 1:
        data_mat = sio.loadmat('./Datasets/IndianPines/Indian_pines_corrected.mat')
        data = data_mat['indian_pines_corrected']
        gt_mat = sio.loadmat('./Datasets/IndianPines/Indian_pines_gt.mat')
        gt = gt_mat['indian_pines_gt']
        # 参数预设
        train_ratio = 0.05  # 训练集比例。注意，训练集为按照‘每类’随机选取
        val_ratio = 0.01 # 测试集比例.注意，验证集选取为从测试集整体随机选取，非按照每类
        class_count = 16  # 样本类别数
        learning_rate = 5e-4  # 学习率
        weight_decay = 2e-5
        max_epoch = 1800  # 迭代次数
        split_height = 1
        split_width = 1
        dataset_name = "indian"  # 数据集名称
        pass

In [17]:
    train_samples_per_class=curr_train_ratio#当定义为每类样本个数时,则该参数更改为训练样本数
    val_samples=class_count

In [18]:
    train_ratio=curr_train_ratio
    if split_height == split_width == 1:
            EDGE = 0
    else:
            EDGE = 5

    cmap = cm.get_cmap('jet', class_count + 1)
    plt.set_cmap(cmap)
    m, n, d = data.shape  # 高光谱数据的三个维度
    n_bands=d

    data = np.reshape(data, [m * n, d])
    minMax = preprocessing.StandardScaler()
    data = minMax.fit_transform(data)
    data = np.reshape(data, [m, n, d])
    

<Figure size 432x288 with 0 Axes>

In [19]:
for curr_seed in Seed_List:
    # step2:随机10%数据作为训练样本。方式：给出训练数据与测试数据的GT
    random.seed(curr_seed)
    gt_reshape = np.reshape(gt, [-1])
    train_rand_idx = []
    val_rand_idx = []

    if samples_type=='ratio':
        #对每一类像素进行编号，取样
        for i in range(class_count):
            idx = np.where(gt_reshape == i + 1)[-1]
            samplesCount = len(idx)
            rand_list = [i for i in range(samplesCount)]  # 用于随机的列表
            rand_idx = random.sample(rand_list, np.ceil(samplesCount * train_ratio).astype('int32'))  # 随机数数量 四舍五入(改为上取整)
            #随机抽取进行train的像素索引
            rand_real_idx_per_class = idx[rand_idx]
            train_rand_idx.append(rand_real_idx_per_class)
        # 总的用于训练的像素
        train_rand_idx = np.array(train_rand_idx)
        train_data_index = []
        for c in range(train_rand_idx.shape[0]):
            a = train_rand_idx[c]
            for j in range(a.shape[0]):
                train_data_index.append(a[j])
        #最终用于训练的索引
        train_data_index = np.array(train_data_index)

        ##将测试集（所有样本，包括训练样本）也转化为特定形式
        train_data_index = set(train_data_index)
        all_data_index = [i for i in range(len(gt_reshape))]
        all_data_index = set(all_data_index)

        # 背景像元的标签
        background_idx = np.where(gt_reshape == 0)[-1]
        background_idx = set(background_idx)
        test_data_index = all_data_index - train_data_index - background_idx

        # 从测试集中随机选取部分样本作为验证集
        val_data_count = int(val_ratio * (len(test_data_index) + len(train_data_index)))  # 验证集数量
        val_data_index = random.sample(test_data_index, val_data_count)
        val_data_index = set(val_data_index)
        test_data_index = test_data_index - val_data_index  # 由于验证集为从测试集分裂出，所以测试集应减去验证集

        # 将训练集 验证集 测试集 整理
        test_data_index = list(test_data_index)
        train_data_index = list(train_data_index)
        val_data_index = list(val_data_index)

                # 获取训练样本的标签图
        train_samples_gt = np.zeros(gt_reshape.shape)
        for i in range(len(train_data_index)):
            # 除参与训练的像素外，其余gt设为0
            train_samples_gt[train_data_index[i]] = gt_reshape[train_data_index[i]]
            pass
        Train_Label=np.reshape(train_samples_gt, [m,n])


        # 获取测试样本的标签图
        test_samples_gt = np.zeros(gt_reshape.shape)
        for i in range(len(test_data_index)):
            test_samples_gt[test_data_index[i]] = gt_reshape[test_data_index[i]]
            pass

        Test_Label = np.reshape(test_samples_gt, [m, n])  # 测试样本图

        # 获取验证集样本的标签图
        val_samples_gt = np.zeros(gt_reshape.shape)
        for i in range(len(val_data_index)):
            val_samples_gt[val_data_index[i]] = gt_reshape[val_data_index[i]]
            pass
        Val_Label=np.reshape(val_samples_gt,[m,n])

        #############将train 和 test 和val 样本标签转化为向量形式###################
        # 训练集
        train_samples_gt = np.reshape(train_samples_gt, [m * n])
        train_samples_gt_vector = np.zeros([m * n, class_count], np.float)
        for i in range(train_samples_gt.shape[0]):
            class_idx = train_samples_gt[i]
            if class_idx != 0:
                temp = np.zeros([class_count])
                temp[int(class_idx - 1)] = 1
                train_samples_gt_vector[i] = temp
        train_samples_gt_vector = np.reshape(train_samples_gt_vector, [m, n, class_count])
        # 测试集
        test_samples_gt = np.reshape(test_samples_gt, [m * n])
        test_samples_gt_vector = np.zeros([m * n, class_count], np.float)
        for i in range(test_samples_gt.shape[0]):
            class_idx = test_samples_gt[i]
            if class_idx != 0:
                temp = np.zeros([class_count])
                temp[int(class_idx - 1)] = 1
                test_samples_gt_vector[i] = temp
        test_samples_gt_vector = np.reshape(test_samples_gt_vector, [m, n, class_count])
        # 验证集
        val_samples_gt = np.reshape(val_samples_gt, [m * n])
        val_samples_gt_vector = np.zeros([m * n, class_count], np.float)
        for i in range(val_samples_gt.shape[0]):
            class_idx = val_samples_gt[i]
            if class_idx != 0:
                temp = np.zeros([class_count])
                temp[int(class_idx - 1)] = 1
                val_samples_gt_vector[i] = temp
        val_samples_gt_vector = np.reshape(val_samples_gt_vector, [m, n, class_count])

        ############制作训练数据和测试数据的gt掩膜.根据GT将带有标签的像元设置为全1向量##############
        # 训练集
        train_label_mask = np.zeros([m * n, class_count])
        temp_ones = np.ones([class_count])
        train_samples_gt = np.reshape(train_samples_gt, [m * n])
        for i in range(m * n):
            if train_samples_gt[i] != 0:
                train_label_mask[i] = temp_ones
        train_label_mask = np.reshape(train_label_mask, [m, n, class_count])

        # 测试集
        test_label_mask = np.zeros([m * n, class_count])
        temp_ones = np.ones([class_count])
        test_samples_gt = np.reshape(test_samples_gt, [m * n])
        for i in range(m * n):
            if test_samples_gt[i] != 0:
                test_label_mask[i] = temp_ones
        test_label_mask = np.reshape(test_label_mask, [m, n, class_count])

        # 验证集
        val_label_mask = np.zeros([m * n, class_count])
        temp_ones = np.ones([class_count])
        val_samples_gt = np.reshape(val_samples_gt, [m * n])
        for i in range(m * n):
            if val_samples_gt[i] != 0:
                val_label_mask[i] = temp_ones
        val_label_mask = np.reshape(val_label_mask, [m, n, class_count])

        # 将数据扩展一维，以满足网络输入需求
        # t1=Train_Label
        # t1[Train_Label>0]=1
        # num=t1.sum()
        # t2=Test_Label
        # t2[Test_Label>0]=1
        # num2=t2.sum()
        Train_Split_Data, Train_Split_GT = SpiltHSI(data, Train_Label, [split_height, split_width], EDGE)
        Test_Split_Data, Test_Split_GT = SpiltHSI(data, Test_Label, [split_height, split_width], EDGE)
        _, patch_height, patch_width, bands = Train_Split_Data.shape
        patch_height -= EDGE * 2
        patch_width -= EDGE * 2





        zero_vector = np.zeros([class_count])
        all_label_mask = np.ones([1, m, n, class_count])  # 设置一个全1的mask，使得网络输出所有分类标签

    train_h=HData((np.transpose(Train_Split_Data,(0,3,1,2)).astype("float32"), Train_Split_GT), None)
    test_h=HData((np.transpose(Test_Split_Data,(0,3,1,2)).astype("float32"), Test_Split_GT), None)
    trainloader=torch.utils.data.DataLoader(train_h)
    testloader=torch.utils.data.DataLoader(test_h)

    use_cuda = torch.cuda.is_available()

In [20]:
model=SSCDNonLModel_gcn(class_count,n_bands,150)

In [21]:
print(model)

SSCDNonLModel_gcn(
  (b1): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (con1): Conv2d(200, 150, kernel_size=(1, 1), stride=(1, 1))
  (s1): Sigmoid()
  (cond1): Conv2d(150, 150, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=150)
  (sd1): Sigmoid()
  (b2): BatchNorm2d(350, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (con2): Conv2d(350, 150, kernel_size=(1, 1), stride=(1, 1))
  (s2): Sigmoid()
  (cond2): Conv2d(150, 25, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=25)
  (sd2): Sigmoid()
  (b4): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (att_map): Self_Attn(
    (query_conv): Conv2d(25, 3, kernel_size=(1, 1), stride=(1, 1))
    (key_conv): Conv2d(25, 3, kernel_size=(1, 1), stride=(1, 1))
    (softmax): Softmax(dim=-1)
  )
  (conv1): GCNConv(1, 12)
  (conv2): GCNConv(12, 25)
  (bcat): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (

In [ ]:
        att_map=self.att_map(xx)
        adj=att_map
        adj[adj<0.5]=0
        adj[adj>=0.5]=1
        edge_index=adj.nonzero().t().contiguous()

In [2]:
import torch
att_map=torch.randn(1,21025,21025)
adj=att_map
adj[adj>=0.5]=1
adj[adj<0.5]=0

In [7]:
print(adj.shape)

torch.Size([21025, 21025])


In [6]:
adj=adj.view(21025,21025)

In [8]:
edge_index=adj.nonzero().t().contiguous()

In [9]:
print(edge_index.shape)

torch.Size([2, 136395699])
